# DECISION TREE Y RANDOM FOREST
---

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import tree
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings('once')

/home/sanfermen/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
df_est = pd.read_csv("../datos/10-bikes_enc_est_mediana.csv", index_col = 0)
df_est.head()

,season,yr,mnth,holiday,weekday,workingday,weathersit,atemp,hum,windspeed,registered
0,0,1,0,0,0,1,2,-0.680818,1.267606,-0.363437,654
1,0,1,0,7,2,2,2,-0.741507,0.480415,0.874656,670
2,0,1,1,7,2,2,4,-1.750344,-1.376017,0.871424,1229
3,1,1,1,7,1,1,4,-1.610886,-0.277410,-0.365545,1454
4,1,1,2,7,1,2,4,-1.505615,-1.378284,0.008358,1518


In [3]:
df_est.isnull().sum()

season        0
yr            0
mnth          0
holiday       0
weekday       0
workingday    0
weathersit    0
atemp         0
hum           0
windspeed     0
registered    0
dtype: int64

In [4]:
# Creamos una clase para poder hacer el decision tree y el random forest más rápido
class Regres_lineal:
    def __init__(self, dataf, col_pred):
        
        self.dataf = dataf
        self.col_pred = col_pred

    def tree_param(self):
        X = self.dataf.drop(self.col_pred, axis = 1)
        y = self.dataf[self.col_pred]

        x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

        arbol = DecisionTreeRegressor(random_state =0)
    
        arbol.fit(x_train, y_train)

        max_feat = np.sqrt(len(x_train.columns))
        max_dep = arbol.tree_.max_depth
        return(max_feat, max_dep)

    def metricas(self, y_te, y_tr, y_te_pred, y_tr_pred, tipo_modelo):
        resultados = {'MAE': [mean_absolute_error(y_te, y_te_pred), mean_absolute_error(y_tr, y_tr_pred)],
                    'MSE': [mean_squared_error(y_te, y_te_pred), mean_squared_error(y_tr, y_tr_pred)],
                    'RMSE': [np.sqrt(mean_squared_error(y_te, y_te_pred)), np.sqrt(mean_squared_error(y_tr, y_tr_pred))],
                    'R2':  [r2_score(y_te, y_te_pred), r2_score(y_tr, y_tr_pred)],
                    "set": ["test", "train"]}
        df = pd.DataFrame(resultados)
        df["modelo"] = tipo_modelo
        return df

    def grid_s_gradient(self, dict, model):
        self.dict = dict
        self.model = model
        
        X = self.dataf.drop(self.col_pred, axis = 1)
        y = self.dataf[self.col_pred]
        x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
        
        gs = GridSearchCV(
            estimator= GradientBoostingRegressor(), 
            param_grid= self.dict, 
            cv=10, 
            verbose=-1,
            return_train_score = True,
            scoring="neg_mean_squared_error")
        gs.fit(x_train, y_train)
        mejor_modelo = gs.best_estimator_
        y_pred_test_dt2 = mejor_modelo.predict(x_test)
        y_pred_train_dt2 = mejor_modelo.predict(x_train)
        dt_results = self.metricas(y_test, y_train, y_pred_test_dt2, y_pred_train_dt2, self.model)
        return mejor_modelo, dt_results

    def grid_s_forest(self, dict, model):
        self.dict = dict
        self.model = model
        
        X = self.dataf.drop(self.col_pred, axis = 1)
        y = self.dataf[self.col_pred]
        x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
        
        gs = GridSearchCV(
            estimator= RandomForestRegressor(), 
            param_grid= self.dict, 
            cv=10, 
            verbose=-1,
            return_train_score = True,
            scoring="neg_mean_squared_error")
        gs.fit(x_train, y_train)
        mejor_modelo = gs.best_estimator_
        y_pred_test_dt2 = mejor_modelo.predict(x_test)
        y_pred_train_dt2 = mejor_modelo.predict(x_train)
        dt_results = self.metricas(y_test, y_train, y_pred_test_dt2, y_pred_train_dt2, self.model)
        return mejor_modelo, dt_results

In [5]:
# Llamamos a la clase
rg_est = Regres_lineal(df_est, "registered")

In [6]:
features_est, depth_est = rg_est.tree_param()

In [7]:
features_est #El número máx será 3

3.1622776601683795

In [8]:
depth_est # La profundidad máxima será 20

20

### Empezamos las diferentes pruebas

In [9]:
param1 = {"max_depth": [16, 18, 20], 
        "max_features": [3,4],
        "min_samples_split": [50, 100],
        "min_samples_leaf": [50,100]} 

In [10]:
best_model1, result1 = rg_est.grid_s_gradient(param1, "GradientBoosting Estand I")

In [11]:
best_model1

GradientBoostingRegressor(max_depth=20, max_features=4, min_samples_leaf=50,
                          min_samples_split=50)

In [12]:
result1

,MAE,MSE,RMSE,R2,set,modelo
0,526.109705,479065.473430,692.145558,0.778064,test,GradientBoosting Estand I
1,485.986996,394377.142099,627.994540,0.841811,train,GradientBoosting Estand I


In [13]:
param2 = {"max_depth": [4,6,8,10], 
        "max_features": [2, 3,4],
        "min_samples_split": [ 40, 50, 60],
        "min_samples_leaf": [10, 20]} 

In [14]:
best_model2, result2 = rg_est.grid_s_gradient(param2, "GradientBoosting Estand II")

In [15]:
best_model2

GradientBoostingRegressor(max_depth=4, max_features=3, min_samples_leaf=10,
                          min_samples_split=50)

In [16]:
result2

,MAE,MSE,RMSE,R2,set,modelo
0,559.590746,505593.020674,711.050646,0.765774,test,GradientBoosting Estand II
1,420.998233,300945.550376,548.585044,0.879288,train,GradientBoosting Estand II


In [17]:
param3 = {"max_depth": [ 12, 14, 16, 18], 
        "max_features": [3,4],
        "min_samples_split": [10, 20, 40],
        "min_samples_leaf": [10, 20, 40]} 

In [18]:
best_model3, result3 = rg_est.grid_s_forest(param3, "RandForest Estand I")

In [19]:
best_model3

RandomForestRegressor(max_depth=16, max_features=4, min_samples_leaf=10,
                      min_samples_split=20)

In [20]:
result3

,MAE,MSE,RMSE,R2,set,modelo
0,596.603578,559894.606376,748.261055,0.740618,test,RandForest Estand I
1,579.464840,531193.775015,728.830416,0.786933,train,RandForest Estand I


In [21]:
param4 = {"max_depth": [12, 14], 
        "max_features": [4,5],
        "min_samples_split": [ 40, 50],
        "min_samples_leaf": [10, 20,]} 

In [22]:
best_model4, result4 = rg_est.grid_s_forest(param4, "RandForest Estand II")

In [23]:
best_model4

RandomForestRegressor(max_depth=14, max_features=5, min_samples_leaf=10,
                      min_samples_split=40)

In [24]:
result4

,MAE,MSE,RMSE,R2,set,modelo
0,600.585119,558207.435763,747.13281,0.741400,test,RandForest Estand II
1,607.068609,581304.765793,762.43345,0.766833,train,RandForest Estand II


### Cargamos el archivo sin estandarizar

In [25]:
df_sinest = pd.read_csv("../datos/09-bikes_enc_sinest_mediana.csv", index_col = 0)
df_sinest.head()

,season,yr,mnth,holiday,weekday,workingday,weathersit,atemp,hum,windspeed,registered
0,0,1,0,0,0,1,2,18.18125,80.5833,10.749882,654
1,0,1,0,7,2,2,2,17.68695,69.6087,16.652113,670
2,0,1,1,7,2,2,4,9.47025,43.7273,16.636703,1229
3,1,1,1,7,1,1,4,10.60610,59.0435,10.739832,1454
4,1,1,2,7,1,2,4,11.46350,43.6957,12.522300,1518


In [26]:
rg_sinest = Regres_lineal(df_sinest, "registered")

In [27]:
features_sinest, depth_sinest = rg_sinest.tree_param()

In [28]:
features_sinest #El número máx será 3

3.1622776601683795

In [29]:
depth_sinest # La profundidad máxima será 20

20

In [30]:
best_model5, result5 = rg_sinest.grid_s_gradient(param1, "GradientBoosting SinEstand I")

In [31]:
result5

,MAE,MSE,RMSE,R2,set,modelo
0,532.844152,493870.493027,702.759200,0.771205,test,GradientBoosting SinEstand I
1,481.928499,383888.735161,619.587552,0.846018,train,GradientBoosting SinEstand I


In [32]:
best_model6, result6 = rg_sinest.grid_s_gradient(param2, "GradientBoosting SinEstand II")

In [33]:
result6

,MAE,MSE,RMSE,R2,set,modelo
0,563.377618,507604.175836,712.463456,0.764843,test,GradientBoosting SinEstand II
1,362.902878,226181.137114,475.585047,0.909276,train,GradientBoosting SinEstand II


In [34]:
best_model7, result7 = rg_sinest.grid_s_forest(param3, "RandForest SinEstand I")

In [35]:
result7

,MAE,MSE,RMSE,R2,set,modelo
0,607.656947,580944.744912,762.197314,0.730866,test,RandForest SinEstand I
1,575.865120,527641.137494,726.389109,0.788358,train,RandForest SinEstand I


In [36]:
best_model8, result8 = rg_sinest.grid_s_forest(param4, "RandForest SinEstand II")

In [37]:
result8

,MAE,MSE,RMSE,R2,set,modelo
0,613.677436,587115.706496,766.234759,0.728007,test,RandForest SinEstand II
1,605.926929,588052.818488,766.846020,0.764126,train,RandForest SinEstand II


In [38]:
param5 = {"max_depth": [22], 
        "max_features": [4],
        "min_samples_split": [8],
        "min_samples_leaf": [8]}

In [39]:
best_model9, result9 = rg_sinest.grid_s_forest(param5, "RandForest SinEstand III")

In [40]:
result9

,MAE,MSE,RMSE,R2,set,modelo
0,578.738671,545173.033054,738.358336,0.747438,test,RandForest SinEstand III
1,546.474265,488840.929397,699.171602,0.803921,train,RandForest SinEstand III


In [41]:
best_model10, result10 = rg_est.grid_s_forest(param5, "RandForest Estand III")

In [42]:
result10

,MAE,MSE,RMSE,R2,set,modelo
0,587.503685,537660.206710,733.253167,0.750918,test,RandForest Estand III
1,545.875880,479000.417186,692.098560,0.807868,train,RandForest Estand III


In [43]:
param6 = {"max_depth": [18,20], 
        "max_features": [3,4],
        "min_samples_split": [8, 15],
        "min_samples_leaf": [8, 15]}

In [44]:
best_model11, result11 = rg_est.grid_s_forest(param6, "RandForest Estand IV")

In [45]:
best_model11

RandomForestRegressor(max_depth=18, max_features=4, min_samples_leaf=8,
                      min_samples_split=8)

In [46]:
result11

,MAE,MSE,RMSE,R2,set,modelo
0,580.862598,531987.698580,729.374868,0.753546,test,RandForest Estand IV
1,544.282163,477650.432116,691.122588,0.808410,train,RandForest Estand IV


In [47]:
best_model12, result12 = rg_est.grid_s_gradient(param5, "GradientBoosting Estand III")

In [48]:
result12

,MAE,MSE,RMSE,R2,set,modelo
0,580.307834,558807.966090,747.534592,0.741121,test,GradientBoosting Estand III
1,106.653767,23397.384584,152.962036,0.990615,train,GradientBoosting Estand III


In [49]:
best_model13, result13 = rg_sinest.grid_s_gradient(param5, "GradientBoosting SinEstand III")

In [50]:
best_model13

GradientBoostingRegressor(max_depth=22, max_features=4, min_samples_leaf=8,
                          min_samples_split=8)

In [51]:
result13

,MAE,MSE,RMSE,R2,set,modelo
0,599.332391,587366.331684,766.398285,0.727891,test,GradientBoosting SinEstand III
1,104.540534,22800.453234,150.998190,0.990855,train,GradientBoosting SinEstand III


In [52]:
param7 = {"max_depth": [18,20], 
        "max_features": [4],
        "min_samples_split": [8, 16],
        "min_samples_leaf": [8]}

In [53]:
best_model14, result14 = rg_est.grid_s_gradient(param7, "GradientBoosting Estand IV")

In [54]:
best_model14

GradientBoostingRegressor(max_depth=20, max_features=4, min_samples_leaf=8,
                          min_samples_split=8)

In [55]:
result14

,MAE,MSE,RMSE,R2,set,modelo
0,610.185468,612689.148202,782.744625,0.716160,test,GradientBoosting Estand IV
1,105.076449,22874.869715,151.244404,0.990825,train,GradientBoosting Estand IV


In [56]:
param17 = {"max_depth": [4], 
        "max_features": [3],
        "min_samples_split": [50],
        "min_samples_leaf": [10]}

best_model15, result15 = rg_est.grid_s_gradient(param17, "GradientBoosting Estand IV")

result15

,MAE,MSE,RMSE,R2,set,modelo
0,552.071159,510032.505087,714.165601,0.763718,test,GradientBoosting Estand IV
1,419.727999,302474.322658,549.976656,0.878675,train,GradientBoosting Estand IV


In [57]:
param17 = {"max_depth": [4], 
        "max_features": [3],
        "min_samples_split": [50],
        "min_samples_leaf": [10]}

best_model16, result16 = rg_sinest.grid_s_gradient(param17, "GradientBoosting SinEstand IV")

result16

,MAE,MSE,RMSE,R2,set,modelo
0,555.076689,487616.514324,698.295435,0.774102,test,GradientBoosting SinEstand IV
1,429.184108,310973.922509,557.650359,0.875265,train,GradientBoosting SinEstand IV
